<img src="https://hms.harvard.edu/themes/harvardmedical/logo.svg" width= "250px"> 


---

# <img src="https://hail.is/docs/devel/hail-logo-cropped.png" width= "50px"> **Workshop**

This notebook is designed to provide a broad overview of Hail's functionality, with emphasis on the functionality to manipulate and query a genetic dataset. Please refer to <https://hail.is/docs/0.2/index.html> for additional information. This sample notebook was generated based on the following: <https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html>. Note the additional functionality of library `plotting.py`, also part of the development tools from the Hail team.

# **Module 1**

## Introduction to `Hail`

Load HAIL and packages

In [ ]:
import hail as hl
import hail.expr.aggregators as agg
hl.init()

In [ ]:
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span
from bokeh.plotting import figure, show, output_file
import pandas as pd
import os , sys, time
import numpy as np
output_notebook()


To learn more about bokeh, look at https://bokeh.pydata.org/en/latest/

In [ ]:
local_path=os.getcwd()
sys.path.append(local_path)
import plotting

---

Load data from the 1K-Genome project


In [ ]:
hl.utils.get_1kg('data/')

Read genetic data into a matrix table.

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt/')

Hail has its own internal data representation, called a [MatrixTable](https://hail.is/docs/0.2/tutorials/09-matrixtable.html)


Speed up by dividing mt into partitions

In [ ]:
CPU = 4
nodes = 1      # 1 node = 1 machine
mt = mt.repartition( 4 * CPU * nodes)

In [ ]:
type(mt)

The `MatrixTable.describe()` method prints all fields in the table and their types, as well as the keys.

In [ ]:
mt.describe()

In [ ]:
list(mt.row)

In [ ]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

To know exactly the number of variants per chromosome and the nature of our SNPs, we can use `summarize_variants()`.

In [ ]:
hl.summarize_variants(mt)

In [ ]:
mt.qual.show()

The [rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.rows) method can be used to get a table with all the row fields in our MatrixTable.  
You can use the `show` method to display the variants.

In [ ]:
mt.AD.show()

To look at the first few genotype calls, we can use [entries](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.entries) along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format. 

Try changing `take` to `show` in the cell below.

In [ ]:
mt.entry.show(5)

In [ ]:
mt.aggregate_rows(hl.agg.count_where(mt.alleles==['A','T']))

In [ ]:
snp_counts = mt.aggregate_rows(
    hl.array(hl.agg.counter(mt.alleles)))
snp_counts

In [ ]:
type(snp_counts)

In [ ]:
sorted(snp_counts, key=lambda x: x[1])

In [ ]:
mt.aggregate_entries(hl.agg.stats(mt.GQ))

In [ ]:
mt.aggregate_entries(
    hl.agg.filter(mt.GT.is_hom_ref(),hl.agg.stats(mt.GQ)))

In [ ]:
hl.agg.stats?


In [ ]:
mt.aggregate_entries(
    hl.agg.filter(~mt.GT.is_hom_ref(),hl.agg.stats(mt.GQ)))

In [ ]:
mt.aggregate_entries(
    hl.agg.filter(mt.GT.is_het(),hl.agg.stats(mt.GQ)))

In [ ]:
p=hl.plot.histogram(mt.GQ, bins=100)

In [ ]:
show(p)

In [ ]:
p=hl.plot.histogram(mt.filter_entries(mt.GT.is_hom_ref()).GQ, bins=100)

In [ ]:
show(p)

In [ ]:
p=hl.plot.histogram(
    mt.filter_entries(mt.GT.is_het_ref()).GQ, 
    bins=100)
show(p)

In [ ]:
p=hl.plot.histogram(
    mt.filter_entries((mt.DP == 10 ) & mt.GT.is_het_ref()).GQ, 
    bins=100)
show(p)

# **Module 2**

## GWAS in 5 steps

Load phenotypic data as table

In [ ]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

Annotations are important in any genetic study. Column fields are where you will store information about sample like phenotypes, ancestry, sex, and covariates.  Let's annotate the columns in our MatrixTable. 

In [ ]:
# Show the first 10 rows of the table
table.show(10)

Notice that the show command only works this way in tables. In matrix tables it is necessary to specify which of the 3 tables we want to show: rows, columns or entries: 

`table.show()` --> Table

`mt.row.alles.show()` --> Matrix Table

In [ ]:
# This is not common not recommended, but one can preview local data using the shell command sh
%%sh
head data/1kg_annotations.txt

We use the `annotate_cols` method to join the table with the MatrixTable containing our dataset.

In [ ]:
mt = mt.annotate_cols(pheno = table[mt.s])

The information from the table is added to the column field of the matrixtable under "pheno".

### 1. QC:

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt.row.describe()

The hardy-weinberg equilibrium (HWE) states that the allele frequency should remain unchanged within a  population. 

Outliers from hwe are identified by a p-value larger than 1e-6.

In [ ]:
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

In [ ]:
mt = hl.sample_qc(mt)

In [ ]:
mt.col.describe()

Control for allele depth (dp_stats) and missingness (call_rate).

In [ ]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))

Check whether the labels homozygous to reference(hom_ref), heterozygous (het), or homozygous variants (hom_var) are indeed correct. 

Calculate number of alternates :

In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)

In [ ]:
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

mt = mt.filter_entries(filter_condition_ab)

For each of the statistics you can filter for outliers. In this example number of singletons (variants that occur once in the dataset).

In [ ]:
stats_singleton   = mt.aggregate_cols(hl.agg.stats(mt.sample_qc.n_singleton))
mt = mt.filter_cols(mt.sample_qc.n_singleton < (stats_singleton.mean + (3 * stats_singleton.stdev)))
mt = mt.filter_cols(mt.sample_qc.n_singleton > (stats_singleton.mean - (3 * stats_singleton.stdev)))

### 2. Population stratification by genetic ancestry

The primary confounder of single‐nucleotide poly- morphism (SNP) to phenotype associations is genetic ancestry. To control for this, we estimate the principal components (PCs) that summarize genetic ancestry to include as covariates in all analyses. 

Filter for common variants to preserve power in the principal component analysis.

In [ ]:
mt_common = mt.filter_rows(mt.variant_qc.AF[1] > 0.05)

The `pca` method produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The `hwe_normalized_pca` method does the same, using HWE-normalized genotypes for the PCA.

In [ ]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(mt_common.GT, k = 10, compute_loadings = True)

In [ ]:
pprint(eigenvalues)

In [ ]:
scores.show(5, width = 100)

Project the scores from the common variants onto the rare variants. The scores will be used to correct for the population stratification in the following analyses.

In [ ]:
mt = mt.annotate_cols(scores = scores[mt_common.s].scores) 

In [ ]:
mt.scores.dtype

Plot the first two PCs

After plotting the PCA, try to click on the population labels on the left. The plot is interactive, this is done through the `plotting.py` library. 

In [ ]:
pca  = plotting.scatter_plot(mt.scores[0], mt.scores[1],
                  label_fields={
                      'Population': mt.pheno.SuperPopulation,
                    'Caffeine': mt.pheno.CaffeineConsumption},
                  title='PCA, first two principal components', 
                  xlabel='PC1', ylabel='PC2')

show(pca)

### 3. Linear regression

Perform linear regression on caffeine consumption and the variants (that are not equal to reference, thus alternates) with covariates: 
- 1.0 is input variable number of alternate alleles, with input variable the genotype dosage derived from the PL field.
- Gender
- Population stratification (population structure) with 10 PCs for genetic ancestry. 

In [ ]:
gwas = hl.linear_regression_rows(
            y = mt.pheno.CaffeineConsumption,
            x = mt.GT.n_alt_alleles(),
            covariates = [1, mt.pheno.isFemale, 
                          mt.scores[0], mt.scores[1], mt.scores[2],
                          mt.scores[3], mt.scores[4], mt.scores[5],
                          mt.scores[6], mt.scores[7], mt.scores[8],
                          mt.scores[9]]) 

Idenitify your top hits:

In [ ]:
gwas_ordered = gwas.order_by(gwas.p_value)

In [ ]:
gwas_ordered.show(10)

### 4. Visualization

Quantile-quantile plot:

Observed against expected p-value to assess inflation. A successful correction for population stratification should bring the observed p-values closer to expected p-values, visualized as a diagonal line.

In [ ]:
qqplot = hl.plot.qq(gwas.p_value)

In [ ]:
show(qqplot)

Manhattan-like plot

In [ ]:
manh = hl.plot.manhattan(gwas.p_value, 
                         title = "Manhattan-like Plot", 
                         size = 4)

In [ ]:
show(manh)

### 5. Multiple testing correction (Bonferroni)

Calculate the Bonferroni corrected P-value cut off. 

In [ ]:
signlevel = 0.05
N = mt.count_rows()
Bonferroni_line = -np.log10(signlevel / N)

In [ ]:
line = Span(location = Bonferroni_line, 
              dimension = "width", 
              line_color = "red", 
              line_width = 1)

In [ ]:
manh.renderers.extend([line])

In [ ]:
show(manh)

---

# **Module 3**

## Variant discovery

The `aggregate` method can be used to aggregate over rows of the table.
`counter` is an aggregation function that counts the number of occurrences of each unique element. 

In [ ]:
pprint(mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation)))

In [ ]:
mt.aggregate_cols(hl.agg.count_where(hl.is_missing(mt.pheno)))

`stats` is an aggregation function that produces some useful statistics about numeric collections. 

In [ ]:
# Extract entries table
entries = mt.entries()

Group by supper population and chromosome, then count heteregeneous variants

In [ ]:
results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))

In [ ]:
results.show(40)

### Rare variants

In [ ]:
# Compute minor allele frequency and generate an annotation column for rare, low frequency and common variants
entries = entries.annotate(maf = hl.cond(entries.info.AF[0]<0.01, "<1%",
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

In [ ]:
# Group by minor allele frequency and hair color
results2 = (entries.group_by(af_bin = entries.maf, purple_hair = entries.pheno.PurpleHair)
      .aggregate(mean_gq = hl.agg.stats(entries.GQ).mean,
                 mean_dp = hl.agg.stats(entries.DP).mean))

In [ ]:
results2.show()

In [ ]:
# Filter rare variants only
rare_vars = entries.filter(entries.maf=="<1%")

In [ ]:
rare_vars.count()

In [ ]:
# why this instruction works 
rare_vars.aggregate(hl.agg.stats(rare_vars.DP))

In [ ]:
# but this one does not work
rare_vars.aggregate(hl.agg.stats(rare_vars.s))
# answer below

In [ ]:
rare_count_per_sample = rare_vars.aggregate((hl.agg.counter(rare_vars.s)))

In [ ]:
rare_count_per_sample

In [ ]:
count_per_sample = entries.aggregate((hl.agg.counter(entries.s)))

In [ ]:
print(type(count_per_sample))
print(str(len(count_per_sample)) + " samples")

In [ ]:
count_per_sample